In [1]:
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.resnet50 import ResNet50
from keras.layers import GlobalAveragePooling2D, Dropout, Dense
from keras.models import Model
from keras import optimizers
import keras.backend as K
import configparser

Using TensorFlow backend.


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
import os

In [3]:
os.chdir("..")

In [4]:
CONFIG_FILEPATH = "./config/default.ini"
CONFIG = configparser.ConfigParser()
CONFIG.read(CONFIG_FILEPATH)

['./config/default.ini']

In [5]:
CONFIG["model"]["metrics"].split(",")

['accuracy']

In [6]:
# Read and split dataset
train_csv = CONFIG["dataset_information"]["training_csv"]
dataset_folder = CONFIG["dataset_information"]["dataset_folder"]
dataset = pd.read_csv(train_csv, dtype=str)

In [7]:
dataset_folder = "../"

'./Train'

In [8]:
train_dev, test = train_test_split(dataset, test_size=0.1)
train, dev = train_test_split(train_dev, test_size=0.1)

In [9]:
len(train), len(test)

(11457, 1415)

In [10]:
img_height, img_width, depth = 256, 256, 3

color_mode = "grayscale" if depth == 1 else "rgb"

num_classes = 5

In [13]:
train.head()

,Filename,Drscore
8935,train_image8937.jpeg,0
591,train_image593.jpeg,0
13918,train_image13920.jpg,2
11212,train_image11214.jpeg,0
7717,train_image7719.jpeg,2


In [11]:
TRAINING_DATA_GENERATOR = ImageDataGenerator(rescale=1./255)
VALIDATION_DATA_GENERATOR = ImageDataGenerator(rescale=1./255)
TEST_DATA_GENERATOR = ImageDataGenerator(rescale=1./255)

In [12]:
TRAINING_DATA = TRAINING_DATA_GENERATOR.flow_from_dataframe(dataframe=train, directory=dataset_folder, x_col="Filename", y_col="Drscore", class_mode="categorical", target_size=(img_height, img_width), color_mode=color_mode)
VALIDATION_DATA = VALIDATION_DATA_GENERATOR.flow_from_dataframe(dataframe=dev, directory=dataset_folder, x_col="Filename", y_col="Drscore", class_mode="categorical", target_size=(img_height, img_width), color_mode=color_mode)
TEST_DATA = TEST_DATA_GENERATOR.flow_from_dataframe(dataframe=test, directory=dataset_folder, x_col="Filename", y_col="Drscore", class_mode="categorical", target_size=(img_height, img_width), color_mode=color_mode)

Found 0 validated image filenames belonging to 0 classes.
Found 0 validated image filenames belonging to 0 classes.
Found 0 validated image filenames belonging to 0 classes.


/home/vinoth/Apps/anaconda/envs/sic/lib/python3.6/site-packages/keras_preprocessing/image/dataframe_iterator.py:273: UserWarning: Found 11457 invalid image filename(s) in x_col="Filename". These filename(s) will be ignored.
  .format(n_invalid, x_col)
/home/vinoth/Apps/anaconda/envs/sic/lib/python3.6/site-packages/keras_preprocessing/image/dataframe_iterator.py:273: UserWarning: Found 1273 invalid image filename(s) in x_col="Filename". These filename(s) will be ignored.
  .format(n_invalid, x_col)
/home/vinoth/Apps/anaconda/envs/sic/lib/python3.6/site-packages/keras_preprocessing/image/dataframe_iterator.py:273: UserWarning: Found 1415 invalid image filename(s) in x_col="Filename". These filename(s) will be ignored.
  .format(n_invalid, x_col)


In [ ]:
base_model = ResNet50(weights= None, include_top=True, input_shape=(img_height,img_width, depth), classes=num_classes)

In [ ]:
base_model.summary()

In [ ]:
sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)

In [ ]:
base_model.compile(loss="categorical_crossentropy", optimizer=sgd, metrics=['accuracy'])

In [ ]:
base_model.fit_generator(generator=TRAINING_DATA, steps_per_epoch=1,
        epochs=1,
        validation_data=VALIDATION_DATA,
        validation_steps=1)

In [ ]:
base_model.save("trained_model.hdf5")
base_model.save_weights("trained_weights.hdf5")

In [ ]:
PREDICTIONS = base_model.predict_generator(generator=TEST_DATA,
                                      steps=1)

In [ ]:
import numpy as np

In [ ]:
Y_PREDICTIONS = np.argmax(PREDICTIONS, axis=1)

In [ ]:
x = 100